<a href="https://colab.research.google.com/github/brysoncar/Home_Sales/blob/main/Home_Sales_Updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=c133a5fa7d280b625ebd90b9f65eab9c00d520456b8b8a8d97c30dec1da81587
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [3]:
pip install findspark

In [4]:
# Import findspark and initialize.
import findspark
findspark.init()

In [5]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [6]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)

home_sales_df = spark.read.option("header", "true").csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep=',')

home_sales_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [7]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView("home_sales")

In [8]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

from pyspark.sql.functions import avg, round

# Filter the DataFrame to include only four-bedroom houses
four_bedroom_df = home_sales_df.filter(home_sales_df['bedrooms'] == 4)

# Group the filtered DataFrame by date_built and calculate the average price
average_price_per_year = four_bedroom_df.groupBy('date_built').agg(round(avg('price'), 2).alias('average_price'))

# Display the result
average_price_per_year.show()


+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2015|    307908.86|
|      2013|    299999.39|
|      2014|    299073.89|
|      2012|    298233.42|
|      2016|    296050.24|
|      2010|    296800.75|
|      2011|     302141.9|
|      2017|    296576.69|
+----------+-------------+



In [9]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

# Filter the DataFrame to include only homes with 3 bedrooms and 3 bathrooms
three_bedroom_three_bathroom_df = home_sales_df.filter((home_sales_df['bedrooms'] == 3) & (home_sales_df['bathrooms'] == 3))

# Group the filtered DataFrame by date_built and calculate the average price
average_price_per_year = three_bedroom_three_bathroom_df.groupBy('date_built').agg(round(avg('price'), 2).alias('average_price'))

# Display the result
average_price_per_year.show()



+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2015|     288770.3|
|      2013|    295962.27|
|      2014|    290852.27|
|      2012|    293683.19|
|      2016|    290555.07|
|      2010|    292859.62|
|      2011|    291117.47|
|      2017|    292676.79|
+----------+-------------+



In [10]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

# Filter the DataFrame to include only homes with specified criteria
filtered_df = home_sales_df.filter((home_sales_df['bedrooms'] == 3) &
                                   (home_sales_df['bathrooms'] == 3) &
                                   (home_sales_df['floors'] == 2) &
                                   (home_sales_df['sqft_living'] >= 2000))

# Group the filtered DataFrame by date_built and calculate the average price
average_price_per_year = filtered_df.groupBy('date_built').agg(round(avg('price'), 2).alias('average_price'))

# Display the result
average_price_per_year.show()


+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2015|    297609.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2012|    307539.97|
|      2016|     293965.1|
|      2010|    285010.22|
|      2011|    276553.81|
|      2017|    280317.58|
+----------+-------------+



In [11]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

from pyspark.sql.functions import avg, round

# Filter the DataFrame to include only expensive homes (price >= $350,000)
expensive_homes_df = home_sales_df.filter(home_sales_df['price'] >= 350000)

# Find the average price of expensive homes
average_price_expensive_homes = expensive_homes_df.groupBy('date_built').agg(round(avg('price'), 2).alias('average_price'))

# Find the "view" rating for the average price of expensive homes
view_rating_for_average_price = average_price_expensive_homes.select(round(avg('average_price'), 2).alias('view_rating'))

# Display the result
view_rating_for_average_price.show()


print("--- %s seconds ---" % (time.time() - start_time))

+-----------+
|view_rating|
+-----------+
|  473769.11|
+-----------+

--- 2.6865928173065186 seconds ---


In [12]:
# Importing necessary functions
from pyspark.sql.functions import count

# Performing value count on the 'view' column
view_counts = home_sales_df.groupBy('view').agg(count('*').alias('count'))

# Displaying the result
view_counts.show()

+----+-----+
|view|count|
+----+-----+
|  31|  607|
|  85|   39|
|  65|   15|
|  53|   20|
|  78|   30|
|  34|  652|
|  81|   33|
|  28|  624|
|  76|   36|
|  26|  662|
|  27|  597|
|  44|  639|
|  12|  589|
|  91|   41|
|  22|  602|
|  93|   33|
|  47|  648|
|   1|  645|
|  52|   23|
|  13|  603|
+----+-----+
only showing top 20 rows



In [13]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

In [14]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [16]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

from pyspark.sql.functions import avg, round

# Filter the DataFrame to include only expensive homes (price >= $350,000)
expensive_homes_df = home_sales_df.filter(home_sales_df['price'] >= 350000)

# Find the average price of expensive homes per view rating
average_price_per_view_rating = expensive_homes_df.groupBy('view').agg(round(avg('price'), 2).alias('average_price'))

# Display the result
average_price_per_view_rating.show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|average_price|
+----+-------------+
|  31|    399856.95|
|  85|   1056336.74|
|  65|    736679.93|
|  53|     755214.8|
|  78|   1080649.37|
|  34|    401419.75|
|  81|   1053472.79|
|  28|    402124.62|
|  76|   1058802.78|
|  26|    401506.97|
|  27|    399537.66|
|  44|    400598.05|
|  12|    401501.32|
|  91|   1137372.73|
|  22|    402022.68|
|  93|   1026006.06|
|  47|     398447.5|
|   1|    401044.25|
|  52|    733780.26|
|  13|    398917.98|
+----+-------------+
only showing top 20 rows

--- 1.0946650505065918 seconds ---


In [1]:
start_time = time.time()

from pyspark.sql.functions import avg, round

# Filter the DataFrame to include only expensive homes (price >= $350,000)
expensive_homes_df = home_sales_df.filter(home_sales_df['price'] >= 350000)

# Find the average price of expensive homes per view rating
average_price_per_view_rating = expensive_homes_df.groupBy('view').agg(round(avg('price'), 2).alias('average_price'))

# Display the result
average_price_per_view_rating.show()

print("--- %s seconds ---" % (time.time() - start_time)

SyntaxError: incomplete input (<ipython-input-1-c30f20ec5781>, line 14)

In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

home_sales_df.write.partitionBy("date_built").parquet("partitioned_home_sales_data")

In [ ]:
#11. Read the formatted parquet data.
partitioned_parquet_data = spark.read.parquet("partitioned_home_sales_data")

In [ ]:
# 12. Create a temporary table for the parquet data.
partitioned_parquet_data.createOrReplaceTempView("partitioned_home_sales_temp_view")


In [17]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime

start_time = time.time()

from pyspark.sql.functions import avg, round

# Filter the DataFrame to include only expensive homes (price >= $350,000)
expensive_homes_df = home_sales_df.filter(home_sales_df['price'] >= 350000)

# Find the average price of expensive homes per view rating
average_price_per_view_rating = expensive_homes_df.groupBy('view').agg(round(avg('price'), 2).alias('average_price'))

# Display the result
average_price_per_view_rating.show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|average_price|
+----+-------------+
|  31|    399856.95|
|  85|   1056336.74|
|  65|    736679.93|
|  53|     755214.8|
|  78|   1080649.37|
|  34|    401419.75|
|  81|   1053472.79|
|  28|    402124.62|
|  76|   1058802.78|
|  26|    401506.97|
|  27|    399537.66|
|  44|    400598.05|
|  12|    401501.32|
|  91|   1137372.73|
|  22|    402022.68|
|  93|   1026006.06|
|  47|     398447.5|
|   1|    401044.25|
|  52|    733780.26|
|  13|    398917.98|
+----+-------------+
only showing top 20 rows

--- 0.7924187183380127 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.

home_sales_df.unpersist()

DataFrame[id: string, date: date, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]

In [ ]:
# 15. Check if the home_sales is no longer cached

is_cached = home_sales_df.is_cached
if not is_cached:
    print("not cached")
else:
    print("cached")

not cached
